# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 20 2022 2:57PM,246871,16,8517952,Sartorius Bioprocess Solutions,Released
1,Sep 20 2022 2:57PM,246871,16,8550853,Sartorius Bioprocess Solutions,Released
2,Sep 20 2022 2:56PM,246870,16,8529999,Sartorius Bioprocess Solutions,Released
3,Sep 20 2022 2:54PM,246869,10,0085995413,ISDIN Corporation,Released
4,Sep 20 2022 2:54PM,246869,10,0085995426,ISDIN Corporation,Released
5,Sep 20 2022 2:54PM,246869,10,0085995437,ISDIN Corporation,Released
6,Sep 20 2022 2:54PM,246869,10,0085995451,ISDIN Corporation,Released
7,Sep 20 2022 2:54PM,246869,10,0085995511,ISDIN Corporation,Released
8,Sep 20 2022 2:54PM,246869,10,0085995510,ISDIN Corporation,Released
9,Sep 20 2022 2:54PM,246869,10,0085995549,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,246867,Released,1
42,246868,Released,19
43,246869,Released,11
44,246870,Released,1
45,246871,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246867,NaN,NaN,1.0
246868,NaN,NaN,19.0
246869,NaN,NaN,11.0
246870,NaN,NaN,1.0
246871,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0.0,27.0,5.0
246770,0.0,1.0,0.0
246785,0.0,1.0,0.0
246789,0.0,0.0,1.0
246811,0.0,39.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0,27,5
246770,0,1,0
246785,0,1,0
246789,0,0,1
246811,0,39,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,0,27,5
1,246770,0,1,0
2,246785,0,1,0
3,246789,0,0,1
4,246811,0,39,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,,27,5
1,246770,,1,
2,246785,,1,
3,246789,,,1
4,246811,,39,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 20 2022 2:57PM,246871,16,Sartorius Bioprocess Solutions
2,Sep 20 2022 2:56PM,246870,16,Sartorius Bioprocess Solutions
3,Sep 20 2022 2:54PM,246869,10,ISDIN Corporation
14,Sep 20 2022 2:52PM,246868,10,Bio-PRF
33,Sep 20 2022 2:51PM,246867,16,"Emersa Waterbox, LLC"
34,Sep 20 2022 2:44PM,246864,10,ISDIN Corporation
40,Sep 20 2022 2:41PM,246863,10,Bio-PRF
41,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc."
45,Sep 20 2022 2:24PM,246861,10,Bio-PRF
46,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 20 2022 2:57PM,246871,16,Sartorius Bioprocess Solutions,,,2
1,Sep 20 2022 2:56PM,246870,16,Sartorius Bioprocess Solutions,,,1
2,Sep 20 2022 2:54PM,246869,10,ISDIN Corporation,,,11
3,Sep 20 2022 2:52PM,246868,10,Bio-PRF,,,19
4,Sep 20 2022 2:51PM,246867,16,"Emersa Waterbox, LLC",,,1
5,Sep 20 2022 2:44PM,246864,10,ISDIN Corporation,,,6
6,Sep 20 2022 2:41PM,246863,10,Bio-PRF,,,1
7,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc.",,,4
8,Sep 20 2022 2:24PM,246861,10,Bio-PRF,,,1
9,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:57PM,246871,16,Sartorius Bioprocess Solutions,2,,
1,Sep 20 2022 2:56PM,246870,16,Sartorius Bioprocess Solutions,1,,
2,Sep 20 2022 2:54PM,246869,10,ISDIN Corporation,11,,
3,Sep 20 2022 2:52PM,246868,10,Bio-PRF,19,,
4,Sep 20 2022 2:51PM,246867,16,"Emersa Waterbox, LLC",1,,
5,Sep 20 2022 2:44PM,246864,10,ISDIN Corporation,6,,
6,Sep 20 2022 2:41PM,246863,10,Bio-PRF,1,,
7,Sep 20 2022 2:36PM,246862,10,"Methapharm, Inc.",4,,
8,Sep 20 2022 2:24PM,246861,10,Bio-PRF,1,,
9,Sep 20 2022 2:04PM,246860,10,"ACI Healthcare USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:57PM,246871,16,Sartorius Bioprocess Solutions,2,,
1,Sep 20 2022 2:56PM,246870,16,Sartorius Bioprocess Solutions,1,,
2,Sep 20 2022 2:54PM,246869,10,ISDIN Corporation,11,,
3,Sep 20 2022 2:52PM,246868,10,Bio-PRF,19,,
4,Sep 20 2022 2:51PM,246867,16,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:57PM,246871,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
1,Sep 20 2022 2:56PM,246870,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Sep 20 2022 2:54PM,246869,10,ISDIN Corporation,11.0,NaN,NaN
3,Sep 20 2022 2:52PM,246868,10,Bio-PRF,19.0,NaN,NaN
4,Sep 20 2022 2:51PM,246867,16,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 2:57PM,246871,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
1,Sep 20 2022 2:56PM,246870,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Sep 20 2022 2:54PM,246869,10,ISDIN Corporation,11.0,0.0,0.0
3,Sep 20 2022 2:52PM,246868,10,Bio-PRF,19.0,0.0,0.0
4,Sep 20 2022 2:51PM,246867,16,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3702801,98.0,12.0,0.0
15,740334,23.0,66.0,18.0
16,2468424,15.0,11.0,0.0
19,493688,7.0,17.0,4.0
20,493613,54.0,24.0,0.0
21,246770,0.0,1.0,0.0
22,493699,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3702801,98.0,12.0,0.0
1,15,740334,23.0,66.0,18.0
2,16,2468424,15.0,11.0,0.0
3,19,493688,7.0,17.0,4.0
4,20,493613,54.0,24.0,0.0
5,21,246770,0.0,1.0,0.0
6,22,493699,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,98.0,12.0,0.0
1,15,23.0,66.0,18.0
2,16,15.0,11.0,0.0
3,19,7.0,17.0,4.0
4,20,54.0,24.0,0.0
5,21,0.0,1.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,98.0
1,15,Released,23.0
2,16,Released,15.0
3,19,Released,7.0
4,20,Released,54.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,18.0,0.0,4.0,0.0,0.0,0.0
Executing,12.0,66.0,11.0,17.0,24.0,1.0,0.0
Released,98.0,23.0,15.0,7.0,54.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,18.0,0.0,4.0,0.0,0.0,0.0
1,Executing,12.0,66.0,11.0,17.0,24.0,1.0,0.0
2,Released,98.0,23.0,15.0,7.0,54.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,18.0,0.0,4.0,0.0,0.0,0.0
1,Executing,12.0,66.0,11.0,17.0,24.0,1.0,0.0
2,Released,98.0,23.0,15.0,7.0,54.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()